# IFT6390 Explainability Report
Authors: Jianan Zhao and Chuanrui Wang
This notebook won't exist without the awesome work of [Hila et al.](https://github.com/hila-chefer/Transformer-Explainability).

In this report we analyze the explainability of the trained Bert Models.



In [1]:
import os
import warnings

# ! Import packages
proj_path = '/home/mila/j/jianan.zhao/scratch/IFT6390_TextCla/'
os.chdir(proj_path)
warnings.filterwarnings('ignore')

from explainability.BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from explainability.BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from models.Bert.config import BertConfig
from models.Bert.trainer import BertTextClaTrainer, TrainingArguments
from types import SimpleNamespace as SN
import numpy as np
from transformers import Trainer
from sklearn.metrics import classification_report
import utils.func as uf
from functools import partial
from explainability.exp_utils import explain_sentence

In [2]:
ckpt = '/home/mila/j/jianan.zhao/scratch/IFT6390_TextCla/output/Bert/ckpts/bert-base-uncased_lr2e-05_bsz128_wd0.01_do0.1_atdo0.1_cla_do0.1_lbs0.0_e8_we060000tokens_freq5'
tokenizer_path = 'data/tokenizer/60000tokens_freq5/vocab.txt'
model = BertForSequenceClassification.from_pretrained(ckpt, num_labels=3).to("cuda")
model.eval()
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
explanations = Generator(model)
classifications = ["NEGATIVE", "NEURAL", "POSITIVE"]

# ! Init Arguments
cf = BertConfig(SN(wandb_name='OFF', process_mode='')).init()
trainer = BertTextClaTrainer(cf=cf)
hf_trainer = Trainer(args=TrainingArguments(per_device_eval_batch_size=680, output_dir='temp')
                     , model=model)

val_x, val_y = trainer.val_x, trainer.val_y
explain = partial(explain_sentence, tokenizer=tokenizer, explanations=explanations, model=model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Bert config: 
{'att_dropout': 0.1, 'base_model': 'microsoft/deberta-v3-base', 'birth_time': '12_18-21_40_23', 'cla_dropout': 0.1, 'dropout': 0.1, 'early_stop': 30, 'epochs': 3, 'eq_batch_size': 64, 'eval_freq_per_epoch': 3, 'gpus': '0', 'label_smoothing_factor': 0.1, 'lr': 2e-05, 'min_frequency': 5, 'model': 'Bert', 'n_class': 3, 'process_mode': '', 'seed': 0, 'vocab_size': 60000, 'wandb_id': '', 'wandb_name': 'OFF', 'warmup_epochs': 0, 'weight_decay': 0.01}
N-GPUs=1
Eq_batch_size = 64, bsz=64, grad_acc_steps=1, ngpus=1


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Check the Classification Results for Possible Samples

In [3]:
# ! Predict
if not os.path.exists(f := 'temp/check_explainability1.pred'):
    pred = hf_trainer.predict(trainer.val_data)
    y_pred = np.argmax(pred.predictions, axis=-1)
    uf.pickle_save(y_pred, f)
else:
    y_pred = uf.pickle_load(f)
print(classification_report(y_pred, val_y))


***** Running Prediction *****
  Num examples = 104032
  Batch size = 680


Saved temp/check_explainability1.pred
              precision    recall  f1-score   support

           0       0.84      0.81      0.82     54150
           1       0.00      0.00      0.00         0
           2       0.80      0.83      0.82     49882

    accuracy                           0.82    104032
   macro avg       0.55      0.55      0.55    104032
weighted avg       0.82      0.82      0.82    104032



In [4]:
correct_pos_text = val_x[np.logical_and(val_y == 2, val_y == y_pred)][:3].tolist()
incorrect_pos_text = val_x[np.logical_and(val_y == 2, val_y != y_pred)][:3].tolist()

correct_neg_text = val_x[np.logical_and(val_y == 0, val_y == y_pred)][:3].tolist()
incorrect_neg_text = val_x[np.logical_and(val_y == 0, val_y != y_pred)][:3].tolist()

correct_neu_text = val_x[np.logical_and(val_y == 1, val_y == y_pred)][:3].tolist()
incorrect_neu_text = val_x[np.logical_and(val_y == 1, val_y != y_pred)][:3].tolist()

# Check the Explainability for Positive Sentiment Examples

In [5]:
for text in correct_pos_text:
    print(f'The POSITIVE text \"{text}\" is CORRECTLY classified!')
    explain(text=text, true_class=2)

The POSITIVE text "today is a great day for ice cream. " is CORRECTLY classified!
[('[CLS]', 0.0), ('today', 0.5007137060165405), ('is', 0.08027686178684235), ('a', 0.03027295507490635), ('great', 1.0), ('day', 0.17694686353206635), ('for', 0.09898936748504639), ('ice', 0.33973345160484314), ('cream', 0.17786864936351776), ('[UNK]', 0.07979793101549149), ('[SEP]', 0.0)]


The POSITIVE text "listening to cubs v dodger on #xm 175 " is CORRECTLY classified!
[('[CLS]', 0.0), ('listening', 1.0), ('to', 0.037858638912439346), ('cubs', 0.11427396535873413), ('v', 0.08824403584003448), ('dodger', 0.20776373147964478), ('on', 0.1129978597164154), ('[UNK]', 0.16584351658821106), ('xm', 0.20297712087631226), ('175', 0.14033278822898865), ('[SEP]', 0.0)]


The POSITIVE text "@tttorrez have a healthy quesadilla tt!wheat tortilla,chicken,low fat cheese&amp;brown rice. sooo good. " is CORRECTLY classified!
[('[CLS]', 0.0), ('[UNK]', 0.7115693688392639), ('tt', 0.15296681225299835), ('##tor', 0.16616784036159515), ('##rez', 0.03299146890640259), ('have', 0.0), ('a', 0.23784799873828888), ('healthy', 0.48145732283592224), ('quesadilla', 0.467732697725296), ('tt', 0.15877710282802582), ('[UNK]', 0.08027783036231995), ('wheat', 0.3331206738948822), ('tortilla', 0.44829294085502625), ('[UNK]', 0.17090249061584473), ('chicken', 0.3160664141178131), ('[UNK]', 0.23299236595630646), ('low', 0.14526893198490143), ('fat', 0.49762654304504395), ('cheese', 0.6040682792663574), ('[UNK]', 0.17821674048900604), ('amp', 0.5940009951591492), ('[UNK]', 0.24363656342029572), ('brown', 0.9972752928733826), ('rice', 0.6559721231460571), ('[UNK]', 0.2550157308578491), ('sooo', 0.49838894605636597), ('good', 1.0), ('[UNK]', 0.2979375422000885), ('[SEP]', 0.0238273

In [6]:
for text in incorrect_pos_text:
    print(f'The POSITIVE text \"{text}\" is INCORRECTLY classified!')
    explain(text=text, true_class=2)

The POSITIVE text "gotta have dinner  baked chicken + mash " is INCORRECTLY classified!
[('[CLS]', -0.0), ('gotta', -1.0), ('have', -0.681205153465271), ('dinner', -0.4537869095802307), ('baked', -0.03966233879327774), ('chicken', -0.9628228545188904), ('[UNK]', -0.08694732189178467), ('mash', -0.0), ('[SEP]', -0.258801132440567)]


The POSITIVE text "this is the 2nd the weho firefighters were at my building in the past 2 days for no reason, come back " is INCORRECTLY classified!
[('[CLS]', -0.0), ('this', -0.04803115874528885), ('is', -0.012219483964145184), ('the', -0.015593075193464756), ('2nd', -0.05951043590903282), ('the', -0.03508628159761429), ('we', -0.06320091336965561), ('##ho', -0.05887116491794586), ('fire', -0.040988434106111526), ('##fighter', -0.0), ('##s', -0.034905143082141876), ('were', -0.28303834795951843), ('at', -0.19825507700443268), ('my', -0.18858106434345245), ('building', -0.10103552788496017), ('in', -0.10065962374210358), ('the', -0.062442731112241745), ('past', -0.097313791513443), ('2', -0.28186464309692383), ('days', -0.18891890347003937), ('for', -0.07019972801208496), ('no', -1.0), ('reason', -0.11318383365869522), ('[UNK]', -0.01459773164242506), ('come', -0.13684214651584625), ('back', -0.3754879832267761), ('[SEP]', -0.05659591406583786)]


The POSITIVE text "just woke up  wavos soon havent been there in ageeees." is INCORRECTLY classified!
[('[CLS]', -0.0), ('just', -0.035143882036209106), ('woke', -0.0), ('up', -0.038288664072752), ('wa', -0.22078931331634521), ('##vo', -0.021460114046931267), ('##s', -0.01290530152618885), ('soon', -0.01429593376815319), ('havent', -1.0), ('been', -0.38146358728408813), ('there', -0.043970417231321335), ('in', -0.06801798194646835), ('age', -0.018297575414180756), ('##eees', -0.12439437955617905), ('[UNK]', -0.02558755874633789), ('[SEP]', -0.07576799392700195)]


# Check the Explainability for Negative Sentiment Examples

In [7]:
for text in correct_neg_text:
    print(f'The NEGATIVE text \"{text}\" is CORRECTLY classified!')
    explain(text=text, true_class=0)

The NEGATIVE text "@jenniwheller you're out of luck. it's gloomy today. " is CORRECTLY classified!
[('[CLS]', -0.0), ('[UNK]', -0.07120363414287567), ('jenni', -0.015429462306201458), ('##wh', -0.00811453815549612), ('##eller', -0.015611394308507442), ('you', -0.03692779317498207), ('[UNK]', -0.03291663900017738), ('re', -0.022481318563222885), ('out', -0.11826583743095398), ('of', -0.0), ('luck', -0.07619883120059967), ('[UNK]', -0.04450193792581558), ('it', -0.005917740520089865), ('[UNK]', -0.041668664664030075), ('s', -0.09507682919502258), ('gloomy', -1.0), ('today', -0.2217911183834076), ('[UNK]', -0.019800398498773575), ('[SEP]', -0.06440427899360657)]


The NEGATIVE text "guess what...i had this really cool idea...but when i logged on i forgot...o well thats what lack of sleep does 4 ya " is CORRECTLY classified!
[('[CLS]', -0.0), ('guess', -0.03571392223238945), ('what', -0.0024923193268477917), ('[UNK]', -0.0005050164763815701), ('[UNK]', -0.012758474797010422), ('[UNK]', -0.01184551976621151), ('i', -0.14542704820632935), ('had', -0.21682314574718475), ('this', -0.08687455207109451), ('really', -0.15111757814884186), ('cool', -0.04438764229416847), ('idea', -0.002983788261190057), ('[UNK]', -0.008536585606634617), ('[UNK]', -0.00913388654589653), ('[UNK]', -0.009588714689016342), ('but', -1.0), ('when', -0.0027445408049970865), ('i', -0.12476180493831635), ('logged', -0.17279842495918274), ('on', -0.16636045277118683), ('i', -0.11254103481769562), ('forgot', -0.11063733696937561), ('[UNK]', -0.01386707741767168), ('[UNK]', -0.0011579174315556884), ('[UNK]', -0.006647446192800999), ('o', -0.2004510462284088), ('well', -0.00401647947

The NEGATIVE text "@rebeccarudolf i saw it on xmas! everybody was cryin' " is CORRECTLY classified!
[('[CLS]', -0.0), ('[UNK]', -0.0), ('rebecca', -0.006838466972112656), ('##rud', -0.12444256991147995), ('##olf', -0.19930315017700195), ('i', -0.22155992686748505), ('saw', -0.23030607402324677), ('it', -0.01514783687889576), ('on', -0.13600970804691315), ('xmas', -0.03915286064147949), ('[UNK]', -0.060785941779613495), ('everybody', -0.06381490081548691), ('was', -0.16615258157253265), ('cryin', -1.0), ('[UNK]', -0.08833315968513489), ('[SEP]', -0.09309946000576019)]


In [8]:
for text in incorrect_neg_text:
    print(f'The NEGATIVE text \"{text}\" is INCORRECTLY classified!')
    explain(text=text, true_class=0)

The NEGATIVE text "@nicholasjerry92 hi! my name is irene im spanish  your show in madrid was very fantastic   what did happen  to the end of the show??  " is INCORRECTLY classified!
[('[CLS]', 0.0), ('[UNK]', 0.8128106594085693), ('nicholas', 0.14188902080059052), ('##jer', 0.1143178939819336), ('##ry', 0.02252264693379402), ('##92', 0.0261686984449625), ('hi', 0.49980589747428894), ('[UNK]', 0.11137516796588898), ('my', 0.0), ('name', 0.6122390627861023), ('is', 0.06326831132173538), ('irene', 0.7215172052383423), ('im', 0.026461627334356308), ('spanish', 0.3328563868999481), ('your', 0.5047338604927063), ('show', 0.26691457629203796), ('in', 0.05909706652164459), ('madrid', 0.6409784555435181), ('was', 0.029522692784667015), ('very', 0.08724711090326309), ('fantastic', 1.0), ('what', 0.3764449954032898), ('did', 0.11709780991077423), ('happen', 0.3223705589771271), ('to', 0.0685579851269722), ('the', 0.46376797556877136), ('end', 0.205063596367836), ('of', 0.5873023271560669), ('the'

The NEGATIVE text "graham greene... the lawless roads &amp; the power and the glory = my day " is INCORRECTLY classified!
[('[CLS]', 0.0), ('graham', 0.385333776473999), ('green', 0.10241293162107468), ('##e', 0.0), ('[UNK]', 0.09958645701408386), ('[UNK]', 0.200435072183609), ('[UNK]', 0.1610763520002365), ('the', 0.16997016966342926), ('lawl', 0.2623213827610016), ('##ess', 0.2475857436656952), ('roads', 0.3023534119129181), ('[UNK]', 0.1876991093158722), ('amp', 0.3260142207145691), ('[UNK]', 0.22100438177585602), ('the', 0.12451193481683731), ('power', 0.23458853363990784), ('and', 0.2276003658771515), ('the', 0.15114274621009827), ('glory', 1.0), ('[UNK]', 0.1860089749097824), ('my', 0.08122145384550095), ('day', 0.17682267725467682), ('[SEP]', 0.014612553641200066)]


The NEGATIVE text "@catherinecarter     !!!" is INCORRECTLY classified!
[('[CLS]', 0.0), ('[UNK]', 1.0), ('catherine', 0.3165498971939087), ('##carter', 0.5499653220176697), ('[UNK]', 0.2183608114719391), ('[UNK]', 0.5720032453536987), ('[UNK]', 0.45698419213294983), ('[SEP]', 0.0)]


# Check the Explainability for Neural Sentiment Examples

In [9]:
for text in correct_neu_text:
    print(f'The NEURAL text \"{text}\" is CORRECTLY classified!')
    explain(text=text, true_class=1)

In [10]:
for text in incorrect_neu_text:
    print(f'The NEURAL text \"{text}\" is INCORRECTLY classified!')
    explain(text=text, true_class=1)

The NEURAL text "at gwt fireside chat @googleio" is INCORRECTLY classified!
[('[CLS]', 0.0), ('at', 0.431083619594574), ('gw', 0.1938612461090088), ('##t', 0.08894461393356323), ('fires', 0.22468207776546478), ('##ide', 0.07760684192180634), ('chat', 0.30649614334106445), ('[UNK]', 0.2448371946811676), ('google', 1.0), ('##io', 0.4986906349658966), ('[SEP]', 0.0)]


The NEURAL text "@pambeeslyjenna jenna, i went to see night at the museum 2 today and i was so surprised to see three cast members from the office..." is INCORRECTLY classified!
[('[CLS]', 0.0), ('[UNK]', 0.4279656410217285), ('pam', 0.13883456587791443), ('##bees', 0.056511275470256805), ('##ly', 0.014527665451169014), ('##jenna', 0.04743852838873863), ('jenna', 0.1058788150548935), ('[UNK]', 0.06385532766580582), ('i', 0.008936283178627491), ('went', 0.047948744148015976), ('to', 0.01045711524784565), ('see', 0.038777511566877365), ('night', 0.18155540525913239), ('at', 0.02886158600449562), ('the', 0.06336811184883118), ('museum', 0.43712228536605835), ('2', 0.03649699315428734), ('today', 0.03707605227828026), ('and', 0.29274362325668335), ('i', 0.010141056030988693), ('was', 0.09402812272310257), ('so', 0.08560691773891449), ('surprised', 1.0), ('to', 0.007119881454855204), ('see', 0.10895916074514389), ('three', 0.2687070071697235), ('cast', 0.17575056850910187), ('members', 0.30

The NEURAL text "safeway with marvin, janelle, and auntie lhu" is INCORRECTLY classified!
[('[CLS]', 0.0), ('safeway', 0.4629901647567749), ('with', 0.35661834478378296), ('marvin', 0.08735370635986328), ('[UNK]', 0.02135796844959259), ('janelle', 0.25986340641975403), ('[UNK]', 0.029489073902368546), ('and', 0.19786325097084045), ('auntie', 0.7913699746131897), ('lh', 1.0), ('##u', 0.2590881884098053), ('[SEP]', 0.0)]
